# Introduction: This analyses calls variants on the single-cell ATAC data at known donor CHIP mutation regions.
 
### Steps:
1. Create bed file for the known CHIP variants from excel file. 
    keep as hg19 here
    Add +/- 1kb
2. Run cellsnp for pileup near those regions
3. See if any fragments locations overlap the bed file (they should be if there is pileup)
4. Plot cells-by-CHIP mutants AF . Plot bulk for each mutant AF.
5. Using the de-multiplexed cells, how many of the known called variants are found in one donor. Also do the same as 

## Convert txt to bed

In [1]:
config_f = "config.yaml"
outdir=""
minCOUNT = 0.1
minAF = 0

In [2]:
%matplotlib inline
from src.utils.parse_config import read_config_file
from src.utils.data_io import wrap_load_mtx_df

from src.config import ROOT_DIR
from os.path import join, dirname, exists
import os
import pandas as pd
from glob import glob
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import mplh.cluster_help as ch
%load_ext autoreload
%autoreload 2


Project Directory: /data2/mito_lineage


In [3]:
config = read_config_file(config_f)
samples_dict = {ind+1: val for ind, val in enumerate(config["samples"])}
config


{'global': 'PROJECT',
 'variants_f': 'data/processed/gilead_chip_profile/20210222_Croker_CD34_sample_ship_MW.txt',
 'indir': '/data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist',
 'experiment': 'CHIP_april08_2021',
 'genome': 'MTblacklist',
 'ref_fa': '/data2/mito_lineage/data/external/GRCh38_MT_blacklist/fasta/genome.fa',
 'samples': ['Control', 'Flt3l', 'Input']}

### Output

In [4]:
if outdir == "":
    outdir="./output"

if not os.path.exists(outdir):
    os.mkdir(outdir)
    
out_bed = join(outdir, "variants.hg19.1kb.bed")
out_hg19_bed = join(outdir, "variants.hg19.bed")
out_pileup_d = join(outdir,"variants_pileup_hg19")

REGION_VCF = join(outdir, "variants.hg19.vcf")
REGION_VCF_1base = join(outdir, "variants.hg19.1base.vcf")

# for exp in samples_dict.values():
#     if not exists(join(out_pileup_d, exp)):
#         os.mkdir(join(out_pileup_d, exp))
if not exists(join(out_pileup_d)):
    os.mkdir(join(out_pileup_d))

# Aggregate
curr_in = join(config["indir"], "aggregate", "outs")
peaks_bed_f = join(curr_in, "peaks.bed")

# 2. Create bed file for CHIP variants

In [5]:
bed_df = pd.read_csv(join(ROOT_DIR, config["variants_f"]), sep='\t', header=None)
bed_df[1]-=1
bed_df[[0,1,2]].to_csv(out_hg19_bed, sep='\t', header=None, index=False)
meta_df = bed_df.drop([0,1,2], axis=1).copy().rename({3:"REF", 4:"ALT"}, axis=1)

bed_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,chrX,123196932,123196933,T,C,intronic,STAG2,NaN,NaN,6065,34188,0.177401,G13,10386
1,chr12,12022888,12022889,T,C,exonic,ETV6,nonsynonymous SNV,ETV6:NM_001987:exon5:c.T995C:p.I332T,6525,19085,0.341892,G14,14263
2,chr2,25505475,25505476,C,T,exonic,DNMT3A,synonymous SNV,"DNMT3A:NM_001320892:exon4:c.G282A:p.E94E,DNMT3...",1760,15334,0.114778,G14,14263
3,chr7,148544302,148544303,T,G,exonic,EZH2,nonsynonymous SNV,"EZH2:NM_001203247:exon2:c.A88C:p.K30Q,EZH2:NM_...",48,24238,0.001980,S03,RB1070
4,chr4,106196818,106196819,G,T,exonic,TET2,nonsynonymous SNV,TET2:NM_001127208:exon11:c.G5152T:p.V1718L,14327,37742,0.379604,S03,RB1070
5,chrX,76890276,76890277,A,C,intronic,ATRX,NaN,NaN,44,14828,0.002967,S03,RB1070
6,chrX,44950066,44950067,G,A,exonic,KDM6A,nonsynonymous SNV,"KDM6A:NM_001291418:exon24:c.G3599A:p.R1200Q,KD...",56,18167,0.003083,S03,RB1070
7,chrX,44949080,44949081,T,G,exonic,KDM6A,synonymous SNV,"KDM6A:NM_001291418:exon23:c.T3405G:p.P1135P,KD...",78,19177,0.004067,S03,RB1070
8,chr2,25457184,25457185,A,C,exonic,DNMT3A,nonsynonymous SNV,"DNMT3A:NM_001320893:exon18:c.T2246G:p.L749R,DN...",290,19494,0.014876,S03,RB1070
9,chr9,133738339,133738340,A,G,exonic,ABL1,nonsynonymous SNV,"ABL1:NM_005157:exon4:c.A740G:p.K247R,ABL1:NM_0...",23328,71887,0.324509,S10,RB1003


In [6]:
vcf = pd.concat((bed_df, meta_df), axis=1).rename({0:"#CHROM", 1:"POS"}, axis=1)
vcf["ID"] = "."
vcf["QUAL"] = "."
vcf["FILTER"] = "PASS"
vcf["INFO"] = ""
#vcf[["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]].to_csv(REGION_VCF, sep='\t', index=False)


vcf[["#CHROM", "POS", "ID", "REF", "ALT"]].to_csv(REGION_VCF, sep='\t', index=False)


vcf[["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,chrX,123196932,.,T,C,.,PASS,
1,chr12,12022888,.,T,C,.,PASS,
2,chr2,25505475,.,C,T,.,PASS,
3,chr7,148544302,.,T,G,.,PASS,
4,chr4,106196818,.,G,T,.,PASS,
5,chrX,76890276,.,A,C,.,PASS,
6,chrX,44950066,.,G,A,.,PASS,
7,chrX,44949080,.,T,G,.,PASS,
8,chr2,25457184,.,A,C,.,PASS,
9,chr9,133738339,.,A,G,.,PASS,


### Convert bed to (dummy)-vcf  for cellsnp-lite input

In [7]:
# Open a file in write mode to add the comment
# Then close the file and reopen it with pandas in append mode
with open(REGION_VCF, 'w') as f:
    f.write(f'##fileformat=VCFv4.2\n##reference=file://{config["ref_fa"]}\n')
vcf[["#CHROM", "POS", "ID", "REF", "ALT"]].to_csv(REGION_VCF, sep='\t', index=False, mode='a')


## +/- 500bps

In [8]:
bed_df[1]-=500
bed_df[2]+=500
bed_df.to_csv(out_bed, sep='\t', header=None, index=False)

# 2. Run cellsnp
(since bam is separate for each experiment, need to run as that)

### Run with exclFLAG

In [9]:
%matplotlib inline

for exp in samples_dict.values():
    print(exp)
    BAM = join(config["indir"], exp, "outs/possorted_bam.bam")
    BARCODE = join(config["indir"], exp, "outs/filtered_peak_bc_matrix/barcodes.tsv")
    OUT_DIR = join(out_pileup_d, exp+"_1base")
    cmd = f"cellsnp-lite -s {BAM} -b {BARCODE} -O {OUT_DIR} -R {REGION_VCF} --exclFLAG 772  -p 20 --minMAF {minAF} --minCOUNT {minCOUNT} --UMItag None --cellTAG CB "
    print(cmd)
    os.system(cmd)
    AD, DP = wrap_load_mtx_df(OUT_DIR, oth_f=False, prefix="cellSNP.tag",
                         columns=('Variant', 'Cell', 'integer'), inc_af=False,
                         as_dense=True, var_names=False, vcf_prefix="cellSNP.base")

    AF=AD/DP.fillna(0)
    AF
    print((AD>0).sum().sum())
    print((AF>0).sum(axis=1))
    f, ax = plt.subplots(ncols=2, figsize=(10,8))
    sns.heatmap(AF.fillna(0), ax=ax[0], vmin=0)
    ax[0].set_title("Variant Allele Frequency")
    sns.heatmap(DP, ax=ax[1], vmin=0)
    ax[1].set_title("Read Coverage at variant position")
    f.suptitle(exp)

Control
cellsnp-lite -s /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Control/outs/possorted_bam.bam -b /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Control/outs/filtered_peak_bc_matrix/barcodes.tsv -O ./output/variants_pileup_hg19/Control_1base -R ./output/variants.hg19.vcf --exclFLAG 772  -p 20 --minMAF 0 --minCOUNT 0.1 --UMItag None --cellTAG CB 
Flt3l
cellsnp-lite -s /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Flt3l/outs/possorted_bam.bam -b /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Flt3l/outs/filtered_peak_bc_matrix/barcodes.tsv -O ./output/variants_pileup_hg19/Flt3l_1base -R ./output/variants.hg19.vcf --exclFLAG 772  -p 20 --minMAF 0 --minCOUNT 0.1 --UMItag None --cellTAG CB 
Input
cellsnp-lite -s /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Input/outs/pos

## Run 1base just in case

In [10]:
%matplotlib inline

def copy_df(df):
    return df.copy()
def pos_add(df):
    df['POS']+= 1
    return df
def vcf_save(df, fname, config):
    # Then close the file and reopen it with pandas in append mode
    with open(fname, 'w') as f:
        f.write(f'##fileformat=VCFv4.2\n##reference=file://{config["ref_fa"]}\n')
    df[["#CHROM", "POS", "ID", "REF", "ALT"]].to_csv(fname, sep='\t', index=False, mode='a')
    return

vcf_save(vcf.pipe(copy_df).pipe(pos_add),REGION_VCF_1base, config)

for exp in samples_dict.values():
    print(exp)
    BAM = join(config["indir"], exp, "outs/possorted_bam.bam")
    BARCODE = join(config["indir"], exp, "outs/filtered_peak_bc_matrix/barcodes.tsv")
    OUT_DIR = join(out_pileup_d, exp)
    cmd = f"cellsnp-lite -s {BAM} -b {BARCODE} -O {OUT_DIR} -R {REGION_VCF_1base} --exclFLAG 772  -p 20 --minMAF {minAF} --minCOUNT {minCOUNT} --UMItag None --cellTAG CB "
    #shell: "cellsnp-lite -s {input.bam_f} --exclFLAG {params.exclFLAG}  -b {input.barcode_f} -O {output} -p 20 --minMAF {params.minMAF} --minCOUNT {params.minCOUNT} --gzip --chrom chrM --printSkipSNPs --UMItag None --cellTAG CB 2> {log}"#chrM"
    print(cmd)
    os.system(cmd)
    AD, DP = wrap_load_mtx_df(OUT_DIR, oth_f=False, prefix="cellSNP.tag",
                         columns=('Variant', 'Cell', 'integer'), inc_af=False,
                         as_dense=True, var_names=False, vcf_prefix="cellSNP.base")

    AF=AD/DP.fillna(0)
    AF
    print((AD>0).sum().sum())
    print((AF>0).sum(axis=1))
    f, ax = plt.subplots(ncols=2, figsize=(10,8))
    sns.heatmap(AF.fillna(0), ax=ax[0], vmin=0)
    ax[0].set_title("Variant Allele Frequency")
    sns.heatmap(DP, ax=ax[1], vmin=0)
    ax[1].set_title("Read Coverage at variant position")
    f.suptitle(exp)

Control
cellsnp-lite -s /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Control/outs/possorted_bam.bam -b /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Control/outs/filtered_peak_bc_matrix/barcodes.tsv -O ./output/variants_pileup_hg19/Control -R ./output/variants.hg19.1base.vcf --exclFLAG 772  -p 20 --minMAF 0 --minCOUNT 0.1 --UMItag None --cellTAG CB 
Flt3l
cellsnp-lite -s /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Flt3l/outs/possorted_bam.bam -b /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Flt3l/outs/filtered_peak_bc_matrix/barcodes.tsv -O ./output/variants_pileup_hg19/Flt3l -R ./output/variants.hg19.1base.vcf --exclFLAG 772  -p 20 --minMAF 0 --minCOUNT 0.1 --UMItag None --cellTAG CB 
Input
cellsnp-lite -s /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Input/outs/pos

## Run w/out exclflag

In [11]:
%matplotlib inline

for exp in samples_dict.values():
    print(exp)
    BAM = join(config["indir"], exp, "outs/possorted_bam.bam")
    BARCODE = join(config["indir"], exp, "outs/filtered_peak_bc_matrix/barcodes.tsv")
    OUT_DIR = join(out_pileup_d, exp+"_noEXCL")
    cmd = f"cellsnp-lite -s {BAM} -b {BARCODE} -O {OUT_DIR} -R {REGION_VCF}  -p 20 --minMAF {minAF} --minCOUNT {minCOUNT} --UMItag None --cellTAG CB "
    print(cmd)
    os.system(cmd)
    AD, DP = wrap_load_mtx_df(OUT_DIR, oth_f=False, prefix="cellSNP.tag",
                         columns=('Variant', 'Cell', 'integer'), inc_af=False,
                         as_dense=True, var_names=False, vcf_prefix="cellSNP.base")

    AF=AD/DP.fillna(0)
    AF
    print((AD>0).sum().sum())
    print((AF>0).sum(axis=1))
    f, ax = plt.subplots(ncols=2, figsize=(10,8))
    sns.heatmap(AF.fillna(0), ax=ax[0], vmin=0)
    ax[0].set_title("Variant Allele Frequency")
    sns.heatmap(DP, ax=ax[1], vmin=0)
    ax[1].set_title("Read Coverage at variant position")
    f.suptitle(exp)

Control
cellsnp-lite -s /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Control/outs/possorted_bam.bam -b /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Control/outs/filtered_peak_bc_matrix/barcodes.tsv -O ./output/variants_pileup_hg19/Control_noEXCL -R ./output/variants.hg19.vcf  -p 20 --minMAF 0 --minCOUNT 0.1 --UMItag None --cellTAG CB 
Flt3l
cellsnp-lite -s /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Flt3l/outs/possorted_bam.bam -b /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Flt3l/outs/filtered_peak_bc_matrix/barcodes.tsv -O ./output/variants_pileup_hg19/Flt3l_noEXCL -R ./output/variants.hg19.vcf  -p 20 --minMAF 0 --minCOUNT 0.1 --UMItag None --cellTAG CB 
Input
cellsnp-lite -s /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Input/outs/possorted_bam.bam -b /data2/iss

# 3. See if aggregate peak location overlaps with variants

In [12]:
## peaks location
peaks_df = pd.read_csv(peaks_bed_f, sep='\t', header=None)
peaks_df

# CBs = pd.read_csv(join(curr_in, "singlecell.csv"))
# CBs.index= CBs.index+1
# CBs["Condition ID"] = CBs["barcode"].apply(lambda x: x.split("-")[-1])
# CBs["Condition"] = CBs["Condition ID"].astype(int).map(samples_dict)
# CBs.head()

# ## Load peak annotations
# peak_annotations = pd.read_csv(join(curr_in,'peak_annotation.tsv'),sep='\t')
# peak_annotations.index=peak_annotations.index+1
# peak_annotations["gene"] = peak_annotations["gene"].str.upper()
# peak_annotations["Peak"]= peak_annotations.index

# ## Load peaks matrix
# filename = join(curr_in,'filtered_peak_bc_matrix/matrix.mtx')
# peaks_sparse_mtx = pd.read_csv(filename,sep=' ',skiprows=2, header=None).iloc[1:].reset_index(drop=True)
# peaks_sparse_mtx.columns = ["Peak", "Cell", "Count"]



,0,1,2
0,chr1,9920,10570
1,chr1,96482,96714
2,chr1,180685,181125
3,chr1,191595,191964
4,chr1,267775,268263
...,...,...,...
145453,chrY,56763251,56763800
145454,chrY,56825902,56825911
145455,chrY,56826567,56826798
145456,chrY,56828629,56851740


In [13]:
cmd = f"bedtools intersect -a {out_bed} -b {peaks_bed_f} "
print(cmd)
!{cmd}

bedtools intersect -a ./output/variants.hg19.1kb.bed -b /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/aggregate/outs/peaks.bed 
chr12	12022533	12022883	T	C	exonic	ETV6	nonsynonymous SNV	ETV6:NM_001987:exon5:c.T995C:p.I332T	6525	19085	0.341891538	G14	14263
chr9	133737970	133738676	A	G	exonic	ABL1	nonsynonymous SNV	ABL1:NM_005157:exon4:c.A740G:p.K247R,ABL1:NM_007313:exon4:c.A797G:p.K266R	23328	71887	0.324509299	S10	RB1003
chr7	101740809	101741194	C	G	exonic	CUX1	nonsynonymous SNV	CUX1:NM_001202544:exon4:c.C304G:p.R102G,CUX1:NM_001202546:exon4:c.C241G:p.R81G,CUX1:NM_001202543:exon5:c.C352G:p.R118G,CUX1:NM_001913:exon5:c.C352G:p.R118G,CUX1:NM_181500:exon5:c.C352G:p.R118G,CUX1:NM_181552:exon5:c.C319G:p.R107G	5602	14098	0.39736132799999996	S10	RB1003
chr17	7577867	7578868	C	A	intronic	TP53			9161	40848	0.224270466	S10	RB1003
chr4	55561884	55562334	A	T	exonic	KIT	stopgain	KIT:NM_000222:exon2:c.A232T:p.K78X,KIT:NM_001093772:exon2:c.A232T:p.K78X	44	27

In [14]:
cmd = f"bedtools intersect -wo -a {out_hg19_bed} -b {peaks_bed_f} "
print(cmd)
!{cmd}

bedtools intersect -wo -a ./output/variants.hg19.bed -b /data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/aggregate/outs/peaks.bed 
chr9	133738339	133738340	chr9	133737970	133738676	1
chr17	7578367	7578368	chr17	7577701	7582561	1
chr17	7578367	7578368	chr17	7577701	7582561	1


In [23]:
cmd = f'find {outdir} -name "*AD.mtx" -exec wc -l {{}} \;'
!{cmd}
#!find output/ -name "*AD.mtx" -exec wc -l {} \;

6 ./output/variants_pileup/Control/cellSNP.tag.AD.mtx
6 ./output/variants_pileup/Flt3l/cellSNP.tag.AD.mtx
4 ./output/variants_pileup/Input/cellSNP.tag.AD.mtx
11 ./output/variants_pileup/Flt3l_1base/cellSNP.tag.AD.mtx
6 ./output/variants_pileup/Control_noEXCL/cellSNP.tag.AD.mtx
4 ./output/variants_pileup/Input_1base/cellSNP.tag.AD.mtx
4 ./output/variants_pileup/Input_noEXCL/cellSNP.tag.AD.mtx
7 ./output/variants_pileup/Control_1base/cellSNP.tag.AD.mtx
11 ./output/variants_pileup/Flt3l_noEXCL/cellSNP.tag.AD.mtx
5 ./output/variants_pileup_hg19/Control/cellSNP.tag.AD.mtx
6 ./output/variants_pileup_hg19/Flt3l/cellSNP.tag.AD.mtx
3 ./output/variants_pileup_hg19/Input/cellSNP.tag.AD.mtx
3 ./output/variants_pileup_hg19/Flt3l_1base/cellSNP.tag.AD.mtx
3 ./output/variants_pileup_hg19/Control_noEXCL/cellSNP.tag.AD.mtx
3 ./output/variants_pileup_hg19/Input_1base/cellSNP.tag.AD.mtx
3 ./output/variants_pileup_hg19/Input_noEXCL/cellSNP.tag.AD.mtx
3 ./output/variants_pileup_hg19/Control_1base/cellSNP.ta

In [24]:
cmd = f'find {outdir} -name "*DP.mtx" -exec wc -l {{}} \;'
!{cmd}
#!find output/ -name "*DP.mtx" -exec wc -l {} \;

6 ./output/variants_pileup/Control/cellSNP.tag.DP.mtx
6 ./output/variants_pileup/Flt3l/cellSNP.tag.DP.mtx
4 ./output/variants_pileup/Input/cellSNP.tag.DP.mtx
15 ./output/variants_pileup/Flt3l_1base/cellSNP.tag.DP.mtx
9 ./output/variants_pileup/Control_noEXCL/cellSNP.tag.DP.mtx
8 ./output/variants_pileup/Input_1base/cellSNP.tag.DP.mtx
7 ./output/variants_pileup/Input_noEXCL/cellSNP.tag.DP.mtx
9 ./output/variants_pileup/Control_1base/cellSNP.tag.DP.mtx
15 ./output/variants_pileup/Flt3l_noEXCL/cellSNP.tag.DP.mtx
5 ./output/variants_pileup_hg19/Control/cellSNP.tag.DP.mtx
7 ./output/variants_pileup_hg19/Flt3l/cellSNP.tag.DP.mtx
5 ./output/variants_pileup_hg19/Input/cellSNP.tag.DP.mtx
24 ./output/variants_pileup_hg19/Flt3l_1base/cellSNP.tag.DP.mtx
4 ./output/variants_pileup_hg19/Control_noEXCL/cellSNP.tag.DP.mtx
6 ./output/variants_pileup_hg19/Input_1base/cellSNP.tag.DP.mtx
6 ./output/variants_pileup_hg19/Input_noEXCL/cellSNP.tag.DP.mtx
4 ./output/variants_pileup_hg19/Control_1base/cellSNP.t

In [41]:
%matplotlib inline
print((AD>0).sum().sum())
print((AF>0).sum(axis=1))
f, ax = plt.subplots(ncols=2, figsize=(10,8))
sns.heatmap(AF.fillna(0), ax=ax[0], vmin=0)
sns.heatmap(DP, ax=ax[1], vmin=0)

0

## plot